In [1]:
%matplotlib notebook

In [2]:
import pathlib
import numpy as np
import matplotlib.dates
import matplotlib.pyplot as plt
import sunpy.net.attrs
import sunpy.net.dataretriever.attrs.goes
import sunpy.timeseries

In [3]:
result = sunpy.net.Fido.search(
    sunpy.net.attrs.Time(
        start='2022-04-05 04:00',
        end='2022-04-12 04:00'
    ),
    sunpy.net.attrs.Instrument('XRS'),
    sunpy.net.dataretriever.attrs.goes.SatelliteNumber(16),
)

In [4]:
files = sunpy.net.Fido.fetch(result)

Files Downloaded:   0%|          | 0/8 [00:00<?, ?file/s]

sci_xrsf-l2-flx1s_g16_d20220410_v2-1-0.nc:   0%|          | 0.00/5.04M [00:00<?, ?B/s]

sci_xrsf-l2-flx1s_g16_d20220411_v2-1-0.nc:   0%|          | 0.00/5.04M [00:00<?, ?B/s]

sci_xrsf-l2-flx1s_g16_d20220412_v2-1-0.nc:   0%|          | 0.00/5.04M [00:00<?, ?B/s]

In [5]:
timeseries = sunpy.timeseries.TimeSeries(files, concatenate=True)

In [13]:
fig, ax = plt.subplots(figsize=(12, 4))
timeseries.plot()
formatter = matplotlib.dates.DateFormatter("%m-%d %H:%M")
ax.xaxis.set_major_formatter(formatter)
ax.set_xlabel('date')
fig.savefig('goes.png', dpi=600)

<IPython.core.display.Javascript object>

In [16]:
import scipy.ndimage
ts_array = timeseries.to_dataframe()['xrsb'].to_numpy()
ts_array = np.log10(ts_array)
ts_array = np.nan_to_num(ts_array, nan=np.nanmedian(ts_array))
# ts_array = 2 * (ts_array - np.nanmin(ts_array)) / (np.nanmax(ts_array) - np.nanmin(ts_array)) - 1
kernel_radius = 10000
kernel_diameter = 2 * kernel_radius + 1
# ts_array_mean = np.convolve(ts_array, np.ones(kernel_diameter)/kernel_diameter, mode='valid')
ts_array_avg = scipy.ndimage.median_filter(ts_array, kernel_diameter)

<ipython-input-16-91efddb54c98>:3: RuntimeWarning: invalid value encountered in log10
  ts_array = np.log10(ts_array)


In [17]:
ts_array_norm = ts_array - ts_array_avg
# ts_array_norm = ts_array_norm / (-ts_array_norm.min())
# ts_array_norm = np.sign(ts_array_norm) * np.sqrt(np.abs(ts_array_norm))
# ts_array_norm = np.clip(ts_array_norm, -1, 1)

In [18]:
ts_array.min()

-12.181753

In [19]:
plt.figure()
plt.plot(ts_array)
plt.plot(ts_array_avg)

<IPython.core.display.Javascript object>

In [20]:
plt.figure()
plt.plot(ts_array_norm)

<IPython.core.display.Javascript object>

In [21]:
import IPython.display
IPython.display.Audio(ts_array_norm, rate=20000)